<p style="font-weight:bold;"> <span style="font-size: 36px"> Map Template </span> </p>

This Notebook exports the Data nodes, states, and parameters for the target partition to a spreadsheet file which can be read, modified, and is ready to be re-imported.

After setting the desired target Partition, the Data present in the workspace is ready to be exported. 

# Import dependencies

In [1]:
#!import "../Utils/UtilityMethods"

# Select Initialization

Choose to run the Map Template either with the set of Systemorph data in memory or with the data present in the Database. Uncomment the desired option.

In [1]:
/* The DataSource is configured and connected to real database */
//#!eval-notebook "Database/Configure"
/* The Systemorph set of dimensions + mockdata are dispatched to the unconfigured in-memory DataSource */
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

# Workspace Initialization 

Firstly, the Workspace is initialized with basic dimensions and with Data Nodes. For a list of the dimensions please refer to the DataModel/DataStructure notebook.

In [1]:
Workspace.InitializeFrom(DataSource);

# Map Template: Data Node

## Set Partition

In [1]:
static var partition = new PartitionByReportingNode() { ReportingNode = "CH" };

In [1]:
await Workspace.Partition.SetAsync<PartitionByReportingNode>( partition );

## Export

In [1]:
await Export.ToExcel("DataNodes")
    .WithSource(Workspace)
    .PortfolioConfiguration<ReinsurancePortfolio>()
    .PortfolioConfiguration<InsurancePortfolio>()
    .GroupofContractConfiguration<GroupOfReinsuranceContract>(typeof(ReinsurancePortfolio))
    .GroupofContractConfiguration<GroupOfInsuranceContract>(typeof(InsurancePortfolio))
    .MainTabConfigurationWoScenario(partition)
.ExecuteAsync()

# Map Template: Data Node State

## Set Partition

In [1]:
static var partition = new PartitionByReportingNodeAndPeriod() { ReportingNode = "CH", Year = 2020, Month = 12 };

In [1]:
await Workspace.Partition.SetAsync<PartitionByReportingNode>( new PartitionByReportingNode() { ReportingNode = partition.ReportingNode } );

## Export

In [1]:
var dataNodeStates = (await Workspace.Query<DataNodeState>()
                        .Where(x => (x.Year == partition.Year && x.Month <= partition.Month) || x.Year < partition.Year).ToArrayAsync())
                        .GroupBy(x => x.DataNode)
                        .Select(x => x.OrderByDescending(y => y.Year).ThenByDescending(y => y.Month))
                        .Select(x => x.Last())
                        .ToArray();

In [1]:
await Export.ToExcel("DataNodeState")
    .WithSource(Workspace)
    .StateEnumConfiguration() 
    .DataNodeStateConfiguration(dataNodeStates)
    .MainTabConfigurationWoScenario(partition)
.ExecuteAsync()

# Map Template: Data Node Parameter

## Set Partition

In [1]:
static var partition = new PartitionByReportingNodeAndPeriod() { ReportingNode = "CH", Year = 2020, Month = 12 };

In [1]:
await Workspace.Partition.SetAsync<PartitionByReportingNode>( new PartitionByReportingNode() { ReportingNode = partition.ReportingNode } );

## Export

In [1]:
var dataNodeParameters = (await Workspace.Query<DataNodeParameter>()
                        .Where(x => (x.Year == partition.Year && x.Month <= partition.Month) || x.Year < partition.Year).ToArrayAsync())
                        .GroupBy(x => x.GetType().Name)
                        .ToDictionary(x => x.Key, 
                                      x => x.GroupBy(y => y.DataNode)
                                            .Select(y => y.OrderByDescending(z => z.Year).ThenByDescending(z => z.Month))
                                            .Select(y => y.Last())
                                            .ToArray() );

In [1]:
await Export.ToExcel("DataNodeParameter")
    .WithSource(Workspace)
    .DataNodeParameterConfiguration(dataNodeParameters)
    .MainTabConfiguration(partition)
.ExecuteAsync()